<a href="https://colab.research.google.com/github/haidershoaib98/TGCN-Reproduce/blob/main/6322_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Installs

In [ ]:
!apt install libspatialindex-dev
!pip install osmnx

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-dev is already the newest version (1.8.5-5).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [ ]:
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning
import pytorch_lightning as pl

  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-g7es1m3w
  Running command git clone -q https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-g7es1m3w
  Running command git submodule update --init --recursive -q
  From https://github.com/PyTorchLightning/lightning-tutorials
   * branch            290fb466de1fcc2ac6025f74b56906592911e856 -> FETCH_HEAD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached pyDeprecate-0.3.2-py3-none-any.whl (10 kB)
  Using cached typing_extensions-4.1.1-py3-none-any.whl (26 kB)
  Using cached torchmetrics-0.7.3-py3-none-any.whl (398 kB)
  Using cached PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
  Using cached fsspec-2022.3.0-py3-none-any.whl (136 kB)
     |████████████████████████████████| 1.1 MB 5.1 MB/s 
     |██████████████████████████████

In [7]:
# Import Python libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as sp
import h5py
import pickle
import torch

# Dataset Setup

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
ls

MyDrive/  Othercomputers/  Shareddrives/


In [11]:
!git clone https://github.com/haidershoaib98/TGCN-Reproduce.git

fatal: could not create work tree dir 'TGCN-Reproduce': Operation not supported


Haider's Drive (we will connect with github later to make this easier)

In [14]:
# Los Loop
f = pd.read_csv('/content/drive/My Drive/TGCN-Reproduce/data/los_speed.csv')
f_adj = pd.read_csv('/content/drive/My Drive/TGCN-Reproduce/data/los_adj.csv')


In [15]:
los_speed = pd.DataFrame(f)
los_speed_array = los_speed.to_numpy()

In [16]:
los_speed_array.shape

(2016, 207)

# T-GCN Model

# Training

# Results